In [ ]:
import Data_Editing_Helpers as DEH
import Classifier as CLS
import Regressor as RGS
import pandas as pd
import numpy as np
import torch.nn as nn
import joblib
import torch
import time
import re
import os
import nltk
import pickle
import dill
from sklearn.metrics import accuracy_score, r2_score
from sklearn.ensemble import StackingRegressor, StackingClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from torch.nn.utils.rnn import pad_sequence

nltk.download('stopwords')


In [ ]:
# Check GPU availability
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:

## Loading ##
test = pd.read_csv("Data/test.csv")
train = pd.read_csv("Data/train.csv")

In [ ]:
y_name = 'Sentiment1' # What you're trying to predict
x_name = 'TonyID' # User id. Drop this column

# Set this to True if you want to run regression models, False for classification models
is_regression = False

In [ ]:
# Display basic info about datasets
train_info = train.info()
test_info = test.info()

# Display first few rows
train_head = train.head()
test_head = test.head()

# Check for missing values
missing_values_train = train.isnull().sum()
missing_values_test = test.isnull().sum()

# Summary statistics
train_description = train.describe()
test_description = test.describe()

train_info, test_info, train_head, test_head, missing_values_train, missing_values_test, train_description, test_description


In [ ]:
## Wrangling ##
#Future implementation will remove map_seasons and convert_strings_to_ascii
train, test = DEH.map_seasons(train, test)

#train = DEH.convert_strings_to_ascii(train)
#test = DEH.convert_strings_to_ascii(test)
train, test = DEH.dropUnusedColumns(train, test, y_name, x_name)
train = DEH.remove_blank_rows(train, y_name)
train, test = DEH.fill_NA(train, test, 0)

In [ ]:
df_train = train.copy()
df_test = test.copy()
print(df_train.head(10))
print(df_test.head(10))

In [ ]:
# Text cleaning function
STOPWORDS = set(stopwords.words('english'))
def clean_text(text):
    text = str(text).lower()
    # Remove URLs, mentions, hashtags, punctuation
    text = re.sub(r"http\S+|@\S+|#\S+|[^a-z\s]", "", text)
    tokens = text.split()
    tokens = [t for t in tokens if t not in STOPWORDS]
    return " ".join(tokens)

# Text features
X_text = df_train['SentimentText']
y = df_train['Sentiment1']


# Encode target labels (+/- to 0/1)
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Apply cleaning
X_text_cleaned = X_text.apply(clean_text)

mask_nonempty = X_text_cleaned.str.strip() != ""
X_text_cleaned = X_text_cleaned[mask_nonempty]
y_encoded = y_encoded[mask_nonempty]

# Keep only classes with at least 2 samples
class_counts = pd.Series(y_encoded).value_counts()
valid_classes = class_counts[class_counts >= 2].index
mask = np.isin(y_encoded, valid_classes)

X_text_cleaned = X_text_cleaned[mask]
y_encoded = y_encoded[mask]


# Save full dataset for PyTorch models
X_full_for_pytorch = X_text_cleaned.copy()
y_full_for_pytorch = y_encoded.copy()


# Train/test split (if needed)
X_train_text, X_test_text, y_train, y_test = train_test_split(
    X_text_cleaned, y_encoded, test_size=0.2, random_state=1103, stratify=y_encoded
)

# Convert text to numeric vectors
vectorizer = TfidfVectorizer(max_features=2000, stop_words='english')
X_train = vectorizer.fit_transform(X_train_text)  # sparse matrix
X_test = vectorizer.transform(X_test_text)  

if not is_regression:
    print("\nEncoding class labels for classification...")
    label_encoder = LabelEncoder()
    y_train = label_encoder.fit_transform(y_train)
    y_test = label_encoder.transform(y_test)

print(X_text_cleaned.head(30))

In [ ]:

print(f"Feature matrix shape: {X_train.shape}")
# Define evaluation function
def evaluate_model(model_func, X_train, y_train, X_test, y_test, model_name, results, is_regression):
    print(f"\nRunning {model_name} ...")
    start_time = time.time()
    model = model_func(X_train, y_train)
    
    predictions = model.predict(X_test)
    
    if is_regression:
        score = r2_score(y_test, predictions) * 100
    else:
        score = accuracy_score(y_test, predictions) * 100
        
    results.append({"model": model_name, "score": score, "model_obj": model})
    
    elapsed = time.time() - start_time
    print(f"{model_name} score: {score:.3f} (Time: {elapsed:.1f}s)")


# Define evaluation function for Pytorch models (ONLY TRAINING)
def evaluate_model_pytorch(model_func, X_train, y_train, X_test, y_test, model_name, results, is_regression):
    print(f"\nRunning {model_name} ...")
    start_time = time.time()

    # Fit model and get vectorizer
    model, vectorizer = model_func(X_train, y_train)

    def ensure_float32(X):
        if X is None:
            return None
        if hasattr(X, "toarray"):  # handle sparse TF-IDF matrices
            X = X.toarray()
        return np.asarray(X, dtype=np.float32)


    elapsed = time.time() - start_time
    print(f"{model_name} finished (Time: {elapsed:.1f}s) – No test set provided")



## Training Models ##
results = []

if is_regression:
    evaluate_model(RGS.decisiontreeRegressor, X_train, y_train, X_test, y_test, "Decision Tree Regressor", results, is_regression)
    evaluate_model(RGS.linearRegressor, X_train, y_train, X_test, y_test, "Linear Regressor", results, is_regression)
    evaluate_model(RGS.ridgeRegressor, X_train, y_train, X_test, y_test, "Ridge Regressor", results, is_regression)
    evaluate_model(RGS.lassoRegressor, X_train, y_train, X_test, y_test, "Lasso Regressor", results, is_regression)
    evaluate_model(RGS.randomForestRegressor, X_train, y_train, X_test, y_test, "Random Forest Regressor", results, is_regression)
    evaluate_model(RGS.gradientBoostingRegressor, X_train, y_train, X_test, y_test, "Gradient Boosting Regressor", results, is_regression)
    evaluate_model(RGS.catBoostRegressor, X_train, y_train, X_test, y_test, "Cat Boost Regressor", results, is_regression)
    evaluate_model(RGS.knnRegressor, X_train, y_train, X_test, y_test, "KNN Regressor", results, is_regression)
    evaluate_model(RGS.xgBoostRegressor, X_train, y_train, X_test, y_test, "XGBoost Regressor", results, is_regression)

else:
    # CPU / fast tree-based models
    #evaluate_model(CLS.decisiontreeClassifier, X_train, y_train, X_test, y_test, "Decision Tree Classifier", results, is_regression)
    #evaluate_model(CLS.extraTreesClassifier, X_train, y_train, X_test, y_test, "ExtraTrees Classifier", results, is_regression)
    #evaluate_model(CLS.adaboostClassifier, X_train, y_train, X_test, y_test, "AdaBoost Classifier", results, is_regression)
    
    # GPU / fast boosted models
    #evaluate_model(CLS.catBoostClassifier, X_train, y_train, X_test, y_test, "CatBoost Classifier", results, is_regression)
    #evaluate_model(CLS.xgBoostClassifier, X_train, y_train, X_test, y_test, "XGBoost Classifier", results, is_regression)
    
    # Lightweight baseline models for stacking/meta-classification
    #evaluate_model(CLS.logisticRegressionClassifier, X_train, y_train, X_test, y_test, "Logistic Regression Classifier", results, is_regression)
    #evaluate_model(CLS.sgdClassifier, X_train, y_train, X_test, y_test, "SGD Classifier", results, is_regression)

    # Just more models
    # train_loss (down) good 
    # valid_acc (up) good
    # valid_loss (down) good

    #evaluate_model_pytorch(CLS.pytorchClassifier, X_full_for_pytorch, y_full_for_pytorch, X_test, y_test, "PyTorch Classifier", results, is_regression)
    #evaluate_model_pytorch(CLS.pytorchDeepClassifier, X_full_for_pytorch, y_full_for_pytorch, X_test, y_test, "PyTorch Deep Classifier", results, is_regression)
    #evaluate_model_pytorch(CLS.pytorchLSTMClassifier, X_full_for_pytorch, y_full_for_pytorch, X_test, y_test, "PyTorch LSTM Classifier", results, is_regression)
    #Trains no issue but when loaded it bloats ram causing crashes
    #evaluate_model_pytorch(CLS.pytorchCNNClassifier, X_full_for_pytorch, y_full_for_pytorch, X_test, y_test, "PyTorch CNN Classifier", results, is_regression)

    print("Training Logistic Regression as meta-classifier for stacking...")


In [ ]:
# Model mapping (file associations)  
model_mapping = {
    # Regression
    "Decision Tree Regressor": "decisiontreeRegressor.pkl",
    "Linear Regressor": "linearRegressor.pkl",
    "Ridge Regressor": "ridgeRegressor.pkl",
    "Lasso Regressor": "lassoRegressor.pkl",
    "Random Forest Regressor": "randomForestRegressor.pkl",
    "Gradient Boosting Regressor": "gradientBoostingRegressor.pkl",
    "Cat Boost Regressor": "catBoostRegressor.pkl",
    "KNN Regressor": "knnRegressor.pkl",
    "XGBoost Regressor": "xgBoostRegressor.pkl",

    # Classifiers
    "Decision Tree Classifier": "decisiontreeClassifier.pkl",
    "ExtraTrees Classifier": "extraTreesClassifier.pkl",
    "AdaBoost Classifier": "adaModel.pkl",
    "CatBoost Classifier": "catBoostClassifier.pkl",
    "XGBoost Classifier": "xgBoostClassifier.pkl",
    "Logistic Regression Classifier": "logisticRegressionClassifier.pkl",
    "SGD Classifier": "sgdClassifier.pkl",

    # PyTorch models (note: still save as .pth)
    "PyTorch Classifier": "pytorchClassifier.pkl",
    "PyTorch Deep Classifier": "pytorchDeepClassifier.pkl",
    "PyTorch LSTM Classifier": "pytorchLSTMClassifier.pkl",
    "PyTorch CNN Classifier": "pytorchCNNClassifier.pkl",
}


In [ ]:
# Loading Models (Supports dill + local PyTorch classes)
def load_model(filename):
    """
    Loads either:
      - A scikit-learn model (.pkl) saved with pickle or joblib
      - A PyTorch model bundle (.pth) containing {"model_state_dict", "full_model", "vocab"}
    """
    basepath = f'TrainedModels/{filename}'
    weightpath = basepath.replace(".pkl", ".pth")

    # Case 1: scikit-learn model
    if os.path.exists(basepath):
        try:
            with open(basepath, 'rb') as f:
                return joblib.load(f)
        except Exception as e:
            print(f"Error loading {filename} via pickle: {e}")
            try:
                with open(basepath, 'rb') as f:
                    return pickle.load(f)
            except Exception as e2:
                print(f"Failed to load {filename} via joblib: {e2}")
                return None

    # Case 2: PyTorch model (saved using dill)
    elif os.path.exists(weightpath):
        print(f"Detected PyTorch model bundle for {filename}, loading...")
        try:
            bundle = torch.load(
                weightpath,
                map_location='cuda' if torch.cuda.is_available() else 'cpu',
                pickle_module=dill
            )
        except Exception as e:
            print(f"Error loading {filename} with dill: {e}")
            return None

        if isinstance(bundle, dict):
            model = bundle.get("model_class", None)
            vocab = bundle.get("vocab", None)
            if model is not None:
                # instantiate if it's a callable (e.g., a lambda returning the class)
                if model is not None:
                    # Case 1: if model is callable (e.g. lambda returning a class)
                    if callable(model):
                        model = model()

                    # Case 2: if model is still a class type (e.g. <class 'SimpleNet'>)
                    if isinstance(model, type):
                        model = model()

                    if "model_state_dict" in bundle:
                        model.load_state_dict(bundle["model_state_dict"])
                    model.eval()
                    print(f"Successfully loaded full PyTorch model for {filename}")
                    return model, vocab
            else:
                print(f"Bundle for {filename} missing 'model_class' key.")
                return None
        else:
            print(f"Unexpected bundle format for {filename}. Expected dict.")
            return None
    else:
        print(f"Warning: {filename} not found!")
        return None


def encode_and_pad(X_text, vocab):
    """Tokenize, encode using vocab, and pad sequences for PyTorch models."""
    def tokenize(text):
        return re.findall(r"\b\w+\b", text.lower())

    tokenized = [tokenize(t) for t in X_text]
    encoded = [torch.tensor([vocab.get(tok, 1) for tok in toks], dtype=torch.long) for toks in tokenized]
    return pad_sequence(encoded, batch_first=True, padding_value=0)


# Evaluate models
results = []

for model_name, filename in model_mapping.items():
    loaded = load_model(filename)

    # Handle tuple (torch model + vocab)
    if isinstance(loaded, tuple):
        model, vocab = loaded
    else:
        model = loaded
        vocab = None

    if model is None:
        continue

    print(f"\nLoaded {model_name} from disk")

    try:
        # PyTorch models
        if isinstance(model, torch.nn.Module):
            model.eval()
            with torch.no_grad():
                if vocab is not None:
                    # Use raw text input specifically prepared for PyTorch
                    X_test_pytorch = encode_and_pad(X_test_text, vocab)
                    X_tensor = X_test_pytorch.to('cuda' if torch.cuda.is_available() else 'cpu')
                    model = model.to('cuda' if torch.cuda.is_available() else 'cpu')
                    outputs = model(X_tensor)
                    predictions = outputs.argmax(dim=1).cpu().numpy()

                    # Free GPU memory after evaluation
                    del X_tensor, X_test_pytorch, outputs
                    torch.cuda.empty_cache()
                else:
                    raise ValueError(f"Vocab not found for {model_name}, cannot encode X_test.")

        # scikit-learn models
        else:
            predictions = model.predict(X_test)

        # Compute score
        if is_regression:
            score = r2_score(y_test, predictions) * 100
        else:
            score = accuracy_score(y_test, predictions) * 100

        results.append({
            "model": model_name,
            "score": score,
            "model_obj": model,
            "vocab": vocab
        })

        print(f"{model_name} score: {score:.3f}")

        # Free memory for PyTorch model
        if isinstance(model, torch.nn.Module):
            del model
            torch.cuda.empty_cache()

    except Exception as e:
        print(f"Error evaluating {model_name}: {e}")

print(f"\n{len(results)} models evaluated successfully!")


In [ ]:
class TorchModelWrapper:
    def __init__(self, model, vectorizer):
        self.model = model
        self.vectorizer = vectorizer

    def predict(self, X):
        # Vectorize text if needed
        if isinstance(X[0], str):
            X = self.vectorizer.transform(X).toarray().astype('float32')

        with torch.no_grad():
            logits = self.model(torch.tensor(X, dtype=torch.float32))
            probs = torch.softmax(logits, dim=1)
            preds = torch.argmax(probs, dim=1).numpy()
        return preds

In [ ]:
import traceback
# Split sklearn vs PyTorch models
sklearn_models = [m for m in results if "pytorch" not in m["model"].lower()]
pytorch_model_files = [m for m in results if "pytorch" in m["model"].lower()]

print("\nEvaluating Sklearn Models")
def evaluate_sklearn(models, X_test, y_test, is_regression=False):
    for i, model in enumerate(models):
        name = model['model']
        obj = model['model_obj']
        print(f"Evaluating pretrained Sklearn model {i+1}/{len(models)}: {name}")
        try:
            preds = obj.predict(X_test)
            score = r2_score(y_test, preds)*100 if is_regression else accuracy_score(y_test, preds)*100
            model['cv_score'] = score
            print(f"{name} test score: {score:.3f}")
        except Exception as e:
            print(f"Error evaluating {name}: {e}")
            model['cv_score'] = None
    return [m for m in models if m['cv_score'] is not None]

sklearn_models = evaluate_sklearn(sklearn_models, X_test, y_test, is_regression)
# Rebuild pytorch model file list safely
pytorch_model_files = [
    {"model": name, "model_obj": path}
    for name, path in model_mapping.items()
    if "pytorch" in name.lower()
]
# Rebuild pytorch model file list safely
pytorch_model_files = []
for name in model_mapping.keys():
    if "pytorch" in name.lower():
        file_path = model_mapping[name]
        if not file_path.endswith(".pth"):
            file_path = file_path.replace(".pkl", ".pth")
        if os.path.exists(f"TrainedModels/{file_path}"):
            pytorch_model_files.append({
                "model": name,
                "model_obj": f"TrainedModels/{file_path}"
            })
        else:
            print(f"Warning: {file_path} not found in TrainedModels/")


In [ ]:
# Tried to make it so this was not needed but oh well. Running out of time. 
# These come from your embeddings and dataset
# Build embedding matrix first so models can use it directly

def build_embedding_matrix(vocab, glove_path, embedding_dim):
    print("Building embedding matrix...")
    embedding_matrix = torch.zeros(len(vocab), embedding_dim)
    embeddings_index = {}

    # Load GloVe embeddings
    with open(glove_path, encoding="utf8") as f:
        for line in f:
            values = line.strip().split()
            word = values[0]
            vec = np.asarray(values[1:], dtype="float32")
            embeddings_index[word] = vec

    # Fill embedding matrix
    for word, idx in vocab.items():
        if word in embeddings_index:
            embedding_matrix[idx] = torch.tensor(embeddings_index[word])
        else:
            embedding_matrix[idx] = torch.randn(embedding_dim) * 0.1

    print(f"Embedding matrix built: {embedding_matrix.shape}")
    return embedding_matrix


# === define embedding/dataset-related constants ===
num_classes = len(np.unique(y))   # or y_train if you have it split
embed_dim = 100                   # GloVe embedding dimension
hidden_dim = 256                  # same as used during training
glove_path = "./glove.6B.100d.txt"

# Build embedding matrix before defining models
embedding_matrix = build_embedding_matrix(vocab, glove_path, embed_dim)


# === Model architectures ===
class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding = nn.Embedding.from_pretrained(
            embedding_matrix, freeze=False, padding_idx=0
        )
        self.fc = nn.Sequential(
            nn.LayerNorm(embed_dim * 2),
            nn.Linear(embed_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.6),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, X):
        emb = self.embedding(X)
        mean_emb = emb.mean(dim=1)
        max_emb = emb.max(dim=1).values
        features = torch.cat([mean_emb, max_emb], dim=1)  # double embedding info
        return self.fc(features)


class DeepMLPNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding = nn.Embedding.from_pretrained(
            embedding_matrix, freeze=False, padding_idx=0
        )
        self.net = nn.Sequential(
            nn.Linear(embed_dim * 2, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.6),
            nn.Linear(256, num_classes)
        )

    def forward(self, X):
        emb = self.embedding(X)
        mean_emb = emb.mean(dim=1)
        max_emb = emb.max(dim=1).values
        features = torch.cat([mean_emb, max_emb], dim=1)  # double embedding info
        return self.net(features)


# Define LSTM model using embeddings
class LSTMNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding = nn.Embedding.from_pretrained(
            embedding_matrix, freeze=False, padding_idx=0
        )
        # match saved checkpoint from training (128 hidden, bidirectional)
        self.lstm = nn.LSTM(embed_dim, 128, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(128 * 2, num_classes)  # 256 total features
        self.dropout = nn.Dropout(0.3)

    def forward(self, X):
        emb = self.embedding(X)
        lstm_out, (h_n, _) = self.lstm(emb)
        out = torch.cat((h_n[-2], h_n[-1]), dim=1)  # concat final states (bidirectional)
        out = self.dropout(out)
        return self.fc(out)


class CNNNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding = nn.Embedding.from_pretrained(
            embedding_matrix, freeze=False, padding_idx=0
        )
        self.conv = nn.Sequential(
            nn.Conv1d(embed_dim, 64, kernel_size=3, padding=2, dilation=2),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Conv1d(64, 32, kernel_size=3, padding=2, dilation=2),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1)
        )
        self.fc = nn.Linear(32, num_classes)

    def forward(self, X):
        emb = self.embedding(X)        # (batch, seq_len, embed_dim)
        emb = emb.transpose(1, 2)      # (batch, embed_dim, seq_len)
        feat = self.conv(emb).squeeze(-1)
        return self.fc(feat)


In [ ]:
print("\nEvaluating PyTorch Models")
pytorch_scores = []

for idx, m in enumerate(pytorch_model_files):
    name = m["model"]
    filename = m["model_obj"]
    print(f"\nLoading PyTorch model {idx+1}/{len(pytorch_model_files)}: {name}")

    try:
        # Load bundle
        if isinstance(filename, str) and os.path.exists(filename):
            bundle = torch.load(filename, map_location='cpu', weights_only=False)
            state_dict = bundle["model_state_dict"]
            vocab = bundle.get("vocab", None)
            model_class_lambda = bundle.get("model_class", None)

            if model_class_lambda is None:
                print(f"Warning: model_class not saved for {name}, cannot recreate model.")
                continue

            # Call the lambda to get the model class or instance
            if callable(model_class_lambda):
                result = model_class_lambda()
                
                # Check if we got a class or an instance
                if isinstance(result, type):
                    # It's a class, we need to instantiate it
                    print(f"Lambda returned a class for {name}, instantiating...")
                    try:
                        # Try without arguments first
                        model_instance = result()
                    except TypeError:
                        # Needs arguments - infer from vocab and task
                        vocab_size = len(vocab) if vocab else 10000
                        num_classes = len(set(y_test)) if hasattr(y_test, '__iter__') else 2
                        print(f"Instantiating with vocab_size={vocab_size}, num_classes={num_classes}")
                        model_instance = result(vocab_size, num_classes)
                elif isinstance(result, torch.nn.Module):
                    # It's already an instance
                    model_instance = result
                else:
                    raise TypeError(f"Lambda returned unexpected type: {type(result)}")
            elif isinstance(model_class_lambda, torch.nn.Module):
                # Already an instance
                model_instance = model_class_lambda
            else:
                raise TypeError(f"model_class is neither callable nor a Module: {type(model_class_lambda)}")

            # Verify valid model
            if not isinstance(model_instance, torch.nn.Module):
                raise TypeError(f"{name} model_class did not produce a valid torch.nn.Module, got {type(model_instance)}")

            model_instance.load_state_dict(state_dict)

        elif isinstance(filename, torch.nn.Module):
            model_instance = filename
            vocab = m.get("vocab", None)
        else:
            raise ValueError(f"{name} is neither a valid file path nor a model object.")

        # Move model to device
        device = "cuda" if torch.cuda.is_available() else "cpu"
        model_instance = model_instance.to(device)
        model_instance.eval()

        with torch.no_grad():
            print(f"Encoding X_test_text for {name}...")
            X_test_tensor = encode_and_pad(X_test_text, vocab).to(device)
            print(f"Running forward pass for {name}...")
            outputs = model_instance(X_test_tensor)
            preds = outputs.argmax(dim=1).cpu().numpy()

        score = (
            accuracy_score(y_test, preds) * 100
            if not is_regression
            else r2_score(y_test, preds) * 100
        )
        pytorch_scores.append(
            {
                "model": name,
                "score": score,
                "state_dict": model_instance.state_dict(),
                "vocab": vocab,
                "model_class": type(model_instance),
            }
        )
        print(f"{name} test score: {score:.3f}")

        # Free GPU memory
        del model_instance, X_test_tensor, outputs
        torch.cuda.empty_cache()
        print(f"Freed GPU memory for {name}")

    except Exception as e:
        print(f"Failed to load or evaluate {name}: {e}")
        traceback.print_exc()

In [ ]:
import torch
from sklearn.metrics import accuracy_score, r2_score

# manual map of model name -> class
model_name_to_class = {
    "PyTorch Classifier": SimpleNet,
    "PyTorch Deep Classifier": DeepMLPNet,
    "PyTorch LSTM Classifier": LSTMNet,
    "PyTorch CNN Classifier": CNNNet,
}

pytorch_scores = []

for idx, m in enumerate(pytorch_model_files):
    name = m["model"]
    filename = m["model_obj"]
    print(f"\nLoading PyTorch model {idx+1}/{len(pytorch_model_files)}: {name}")

    try:
        # Load bundle
        bundle = torch.load(filename, map_location='cpu', weights_only=False)
        state_dict = bundle['model_state_dict']
        vocab = bundle.get('vocab', None)

        # try to recreate model
        model_cls = model_name_to_class.get(name, None)
        if model_cls is None:
            print(f"Warning: no known class mapping for {name}, skipping.")
            continue

        # Infer the number of classes from the state_dict
        # Look for the final layer's output size
        num_classes = None
        vocab_size = len(vocab) if vocab else 10000
        
        # Check different possible final layer names
        for key in state_dict.keys():
            if 'fc.weight' in key or 'fc.4.weight' in key or 'net.8.weight' in key:
                num_classes = state_dict[key].shape[0]
                print(f"Detected num_classes={num_classes} from state_dict key: {key}")
                break
        
        if num_classes is None:
            print(f"Warning: Could not infer num_classes for {name}, skipping.")
            continue

        # Create model instance with correct parameters
        try:
            model_instance = model_cls(vocab_size, num_classes)
        except TypeError:
            # If the class doesn't accept these parameters, try without
            model_instance = model_cls()
        
        if not isinstance(model_instance, torch.nn.Module):
            raise TypeError(f"{name} model_class mapping did not produce a valid torch.nn.Module")

        # load weights
        model_instance.load_state_dict(state_dict)
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        model_instance.to(device)
        model_instance.eval()

        # forward pass
        with torch.no_grad():
            print(f"Encoding X_test_text for {name}...")
            X_test_tensor = encode_and_pad(X_test_text, vocab)  # keep on CPU first

            all_preds = []
            batch_size = 128  # adjust based on GPU memory

            for i in range(0, len(X_test_tensor), batch_size):
                batch = X_test_tensor[i:i+batch_size].to(device)
                outputs = model_instance(batch)
                preds_batch = outputs.argmax(dim=1).cpu().numpy()
                all_preds.append(preds_batch)

            preds = np.concatenate(all_preds)

        score = accuracy_score(y_test, preds)*100 if not is_regression else r2_score(y_test, preds)*100
        pytorch_scores.append({
            "model": name,
            "score": score,
            "state_dict": state_dict,
            "vocab": vocab,
            "model_class": model_cls
        })
        print(f"{name} test score: {score:.3f}")

        del model_instance, X_test_tensor, outputs
        torch.cuda.empty_cache()
        print(f"Freed GPU memory for {name}")

    except Exception as e:
        print(f"Failed to load or evaluate {name}: {e}")
        import traceback
        traceback.print_exc()

In [ ]:
# Prepare actual test data first
print("\nPreparing actual test data for predictions...")
x_name_values = test[x_name].values
print(f"Sample {x_name} values from test set: {x_name_values[:5]}")
X_test_actual = df_test['SentimentText']
X_test_actual_cleaned = X_test_actual.apply(clean_text)
X_test_actual_cleaned = X_test_actual_cleaned.apply(
    lambda x: "<UNK>" if x.strip() == "" else x
)

# Select top 3 models
top3_sklearn = sorted(sklearn_models, key=lambda x: x['cv_score'], reverse=True)[:3]
top3_pytorch = sorted(pytorch_scores, key=lambda x: x['score'], reverse=True)[:3]

print("\nTop 3 sklearn models:", [m['model'] for m in top3_sklearn])
print("Top 3 PyTorch models:", [m['model'] for m in top3_pytorch])

print("\nTraining Sklearn Stacked Model")

# Vectorize BOTH validation and actual test data
print("Vectorizing validation data...")
X_val_vectorized = vectorizer.transform(X_test_text)  # For scoring
print("Vectorizing actual test data...")
X_test_actual_vectorized = vectorizer.transform(X_test_actual_cleaned)  # For submission

if is_regression:
    sklearn_stack = StackingRegressor(
        estimators=[(m['model'], m['model_obj']) for m in top3_sklearn],
        final_estimator=Ridge(alpha=1.0),
        n_jobs=-1
    )
else:
    sklearn_stack = StackingClassifier(
        estimators=[(m['model'], m['model_obj']) for m in top3_sklearn],
        final_estimator=LogisticRegression(n_jobs=-1),
        n_jobs=-1
    )

print("Fitting sklearn stacked model...")
sklearn_stack.fit(X_train, y_train)

# Get predictions on both validation and actual test
sklearn_val_preds = sklearn_stack.predict(X_val_vectorized)
sklearn_test_preds = sklearn_stack.predict(X_test_actual_vectorized)

sklearn_score = r2_score(y_test, sklearn_val_preds)*100 if is_regression else accuracy_score(y_test, sklearn_val_preds)*100
print(f"Sklearn stacked validation score: {sklearn_score:.3f}")

print("\nTraining PyTorch Stacked Meta Learner")
pytorch_val_matrix = []
pytorch_test_matrix = []

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

for idx, m in enumerate(top3_pytorch):
    model_class = m["model_class"]
    state_dict = m['state_dict']
    vocab = m['vocab']
    
    # Infer num_classes from state_dict
    num_classes = None
    for key in state_dict.keys():
        if 'fc.weight' in key or 'fc.4.weight' in key or 'net.8.weight' in key:
            num_classes = state_dict[key].shape[0]
            break
    
    if num_classes is None:
        num_classes = len(set(y_test)) if hasattr(y_test, '__iter__') else 2
    
    if vocab is not None:
        vocab_size = len(vocab)
    else:
        vocab_size = 20000
    
    try:
        model_instance = model_class(vocab_size, num_classes)
    except TypeError:
        try:
            model_instance = model_class()
        except:
            print(f"Could not instantiate {m['model']}, skipping...")
            continue
    
    model_instance.load_state_dict(state_dict)
    model_instance = model_instance.to(device)
    model_instance.eval()

    with torch.no_grad():
        # Predict on VALIDATION data (for scoring)
        print(f"Encoding validation data for {m['model']}...")
        X_val_tensor = encode_and_pad(X_test_text, vocab)
        
        all_val_preds = []
        batch_size = 512
        
        for i in range(0, len(X_val_tensor), batch_size):
            batch = X_val_tensor[i:i+batch_size]
            if device == 'cuda':
                batch = batch.pin_memory().to(device, non_blocking=True)
            else:
                batch = batch.to(device)
            
            outputs = model_instance(batch)
            preds_batch = outputs.argmax(dim=1).cpu().numpy()
            all_val_preds.append(preds_batch)
            del batch, outputs
        
        val_preds = np.concatenate(all_val_preds)
        pytorch_val_matrix.append(val_preds)
        del X_val_tensor
        
        # Predict on ACTUAL TEST data (for submission)
        print(f"Encoding actual test data for {m['model']}...")
        X_test_actual_tensor = encode_and_pad(X_test_actual_cleaned, vocab)
        
        all_test_preds = []
        
        print(f"Processing {len(X_test_actual_tensor)} test samples...")
        for i in range(0, len(X_test_actual_tensor), batch_size):
            batch = X_test_actual_tensor[i:i+batch_size]
            if device == 'cuda':
                batch = batch.pin_memory().to(device, non_blocking=True)
            else:
                batch = batch.to(device)
            
            outputs = model_instance(batch)
            preds_batch = outputs.argmax(dim=1).cpu().numpy()
            all_test_preds.append(preds_batch)
            del batch, outputs
            
            if (i // batch_size) % 100 == 0:
                print(f"  Processed {i}/{len(X_test_actual_tensor)} samples...")
        
        test_preds = np.concatenate(all_test_preds)
        pytorch_test_matrix.append(test_preds)
        del X_test_actual_tensor

    del model_instance
    if device == 'cuda':
        torch.cuda.empty_cache()
    print(f"Completed predictions for {m['model']}")

pytorch_val_matrix = np.column_stack(pytorch_val_matrix)
pytorch_test_matrix = np.column_stack(pytorch_test_matrix)
print(f"PyTorch validation matrix shape: {pytorch_val_matrix.shape}")
print(f"PyTorch test matrix shape: {pytorch_test_matrix.shape}")

if is_regression:
    pytorch_meta = Ridge(alpha=1.0)
else:
    pytorch_meta = LogisticRegression(max_iter=1000, n_jobs=-1)

print("Fitting PyTorch meta learner on validation data...")
pytorch_meta.fit(pytorch_val_matrix, y_test)

# Get predictions on both
pytorch_val_preds = pytorch_meta.predict(pytorch_val_matrix)
pytorch_test_preds = pytorch_meta.predict(pytorch_test_matrix)

pytorch_score = r2_score(y_test, pytorch_val_preds)*100 if is_regression else accuracy_score(y_test, pytorch_val_preds)*100
print(f"PyTorch stacked validation score: {pytorch_score:.3f}")

print("\nTraining Combined Stack")
combined_val_matrix = np.column_stack([sklearn_val_preds, pytorch_val_preds])
combined_test_matrix = np.column_stack([sklearn_test_preds, pytorch_test_preds])

print(f"Combined validation matrix shape: {combined_val_matrix.shape}")
print(f"Combined test matrix shape: {combined_test_matrix.shape}")

if is_regression:
    combined_meta = Ridge(alpha=1.0)
else:
    combined_meta = LogisticRegression(max_iter=1000, n_jobs=-1)

print("Fitting combined stacked model on validation data...")
combined_meta.fit(combined_val_matrix, y_test)

# Get predictions on both
combined_val_preds = combined_meta.predict(combined_val_matrix)
combined_test_preds = combined_meta.predict(combined_test_matrix)

combined_score = r2_score(y_test, combined_val_preds)*100 if is_regression else accuracy_score(y_test, combined_val_preds)*100
print(f"Combined stacked validation score: {combined_score:.3f}")

# Convert predictions to +/- format
def convert_to_sentiment(predictions):
    """Convert numeric predictions to +/- format"""
    return ['+' if pred == 1 else '-' for pred in predictions]


print("\nSaving tab-delimited submission files...")

# Convert ALL TEST predictions to sentiment
sklearn_test_sentiment = convert_to_sentiment(sklearn_test_preds)
pytorch_test_sentiment = convert_to_sentiment(pytorch_test_preds)
combined_test_sentiment = convert_to_sentiment(combined_test_preds)

# Create DataFrames for ALL THREE submissions
submission_sklearn = pd.DataFrame({
    "ID#": x_name_values,
    "sentiment": sklearn_test_sentiment
})

submission_pytorch = pd.DataFrame({
    "ID#": x_name_values,
    "sentiment": pytorch_test_sentiment
})

submission_combined = pd.DataFrame({
    "ID#": x_name_values,
    "sentiment": combined_test_sentiment
})

# Save ALL THREE tab-delimited text files
submission_sklearn.to_csv("submission_sklearn_stacked.txt", sep='\t', index=False)
submission_pytorch.to_csv("submission_pytorch_stacked.txt", sep='\t', index=False)
submission_combined.to_csv("submission_combined_stacked.txt", sep='\t', index=False)

print("All 3 tab-delimited submission files saved!")
print(f"\nTotal test predictions: {len(x_name_values)}")
print("\nSample of sklearn submission:")
print(submission_sklearn.head(10))
print("\nSample of pytorch submission:")
print(submission_pytorch.head(10))
print("\nSample of combined submission:")
print(submission_combined.head(10))